# Прикладное программное обеспечение
#### Python для извлечения и обработки данных


## API - telegram, vk, бонусы
Социлогия | 1 курс | 3 модуль | Семинар 7

*Авторы: Валентин Бирюков, НИУ ВШЭ*

Это занятие является скорее показательным навыком того, что вы уже можете делать, используя полученные знания, а так же cntl+c-cntl+v с гугла. В ней проведены каркасы и пояснения к некоторым базовым оболочкам кода для взаимодействия с Вк и телеграмм.

## Ссылки

Возпользовавшись источниками ниже данный блокнот можно и вообще пропустить =)
Как и многое другое по программированию все уже возможно где то сделано, нам же предстоит заиспользовать это для своих нужд. Список ниже - ссылки и статьи на основе которых написан данный блокнот.

1) https://python-telegram-bot.org/ - лендинг почти официальной библиотеки модуля для телеграм

2) https://github.com/python-telegram-bot/python-telegram-bot - он же на гитхабе

3) https://habr.com/ru/post/448310/ - туториал по нему, в придачу еще искусственный интеллект.

4) https://github.com/eternnoir/pyTelegramBotAPI - похожая версия

5) https://habr.com/ru/post/442800/ - другой туториал. по похожему модулю, только 


## Быстрый старт

Самая простая и описанная часть. Очень коротко: нужно найти бота @BotFather, написать ему /start, или /newbot, заполнить поля, которые он спросит (название бота и его короткое имя), и получить сообщение с токеном бота и ссылкой на документацию. Токен нужно сохранить, желательно надёжно, так как это единственный ключ для авторизации бота и взаимодействия с ним.

Так же нужно поставить первую библиотеку.

In [14]:
!pip install python-telegram-bot

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
# настройки для бота:
API_TOKEN = '1005315252:AAH3vw40Y3WJnSWanSCjD2ywyS8xqe-pBZ8'
BOT_NAME = 'HseSocExperimentBot'

Напишем простого бота который запоминает имя, а потом отпрашивется немного поучиться.

In [16]:
import logging # модуль для логирования, будет писать нам в консоль о происходящем
from telegram import (ReplyKeyboardMarkup, ReplyKeyboardRemove) # методы для работы ответа "кнопочками"
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler
)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO) 

logger = logging.getLogger(__name__)

In [18]:
GROUP, NAKOP = range(2) # состояния в которых пребывает бот
# GROUP - мы спроисли и получили номер группы
# NAKOP - докапываемся на счет накопа

In [19]:
def start(update, context):
    group_number = [list(map(str, range(191, 199)))] # Варианты доступные для ответа: числа 191-198
    hellow_text = ''' 
Привет! я бот - вышкинец. Я пока только учусь.
Отправь мне /cancel чтобы отправить меня ботать.
    
Расскажи из какой ты группы?
    ''' 
    update.message.reply_text( # отправим сообщение и предложим на выбор варианты из списка
        hellow_text,
        reply_markup=ReplyKeyboardMarkup(group_number, one_time_keyboard=True)
    )
    return GROUP # мы переходим в следующее сосотяние - в нем заберем номер группы и спросим имя

In [24]:
def group(update, context):
    user = update.message.from_user
    group_id = update.message.text
    # Выведем номер группы пользователя себе локально
    logger.info("Получен номер группы пользователя  %s: %s", user.first_name, group_id)
    update.message.reply_text(
        'ох, слышал в группе {} нелегко. Вот какой у тебя накоп?'.format(group_id), 
        reply_markup=ReplyKeyboardRemove() # уберем клавиатуру с прошлого раза, теперь надо написать текстом.
    )

    return NAKOP

In [25]:
import time

def nakop(update, context):
    # 
    user = update.message.from_user
    mark_text = update.message.text
    logger.info("У пользователя %s накоп: %s", user.first_name, mark_text)
    # теперь реализуем такую логику:
    # если пользователь отправил и не особо то число - повторим процедуру еще раз
    # если все таки число - то посмотрим хватает ли на зачет =)
    try: # эта конструкция спасет нас от ошибок.
        mark = float(mark_text) # если это не число и преобразованиие не получится
    except: # то мы провалимся в этот блок
        update.message.reply_text(
            'Это разве накоп? да это и не число то особо. Давай еще раз: какой у тебя накоп?'
        )
        return NAKOP
    
    # если дошли сюда - значит в mark лежит число
    if mark > 3.5:
        update.message.reply_text('Ого! да у тебя уже зачет. А у меня нет. пойду ботать.')
        return ConversationHandler.END
    else:
        update.message.reply_text('Ой, какой то он маленький. Давай я подожду чуть чуть.')
        time.sleep(10)
        update.message.reply_text('А теперь какой?')
        return NAKOP

In [26]:
'''Это системные функции для обработки ошибок и исключений, а также конца разговора. на всякий случай'''

def cancel(update, context):
    user = update.message.from_user
    logger.info("Пользователь %s закончил разговор", user.first_name)
    update.message.reply_text('Ну ладно, пойду ботать',reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END


def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)

In [27]:
def main():
    # создаем канал для обещния с телеграмом. потребуется ключ для общения с ботом
    updater = Updater(API_TOKEN, use_context=True)
    dp = updater.dispatcher

    # конфигурируем какие у нас могут быть состояния
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)], # начинамем с того что описали в функции start
        states={
            GROUP: [MessageHandler(Filters.regex('^19.$'), group)],
            # описываем состояние "группы" - по сути то что описано в функции group
            # фильтруем в этом состоянии все сообщения которые не начинаются на 19*
            NAKOP: [MessageHandler(Filters.text, nakop)]
            # наше второе состояние - докапываем до накопленной оценки
        },
        fallbacks=[CommandHandler('cancel', cancel)]
        # что делать в случае отмены - завершаем диалог функцией cancel
    )

    # собираем и запускаем нашу конфигурацию
    dp.add_handler(conv_handler)
    # log all errors
    dp.add_error_handler(error)
    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [28]:
# эта ячейка зависнет при запуске - потому что пока она запущена
# она контролирует работу бота в телеграме. если она не запущена - бот в телеграме не отвечает
main()

2020-03-16 10:54:33,482 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x10d048828>: Failed to establish a new connection: [Errno 61] Connection refused')': /bot1005315252:AAH3vw40Y3WJnSWanSCjD2ywyS8xqe-pBZ8/getMe
2020-03-16 10:54:33,508 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x10d23cba8>: Failed to establish a new connection: [Errno 61] Connection refused')': /bot1005315252:AAH3vw40Y3WJnSWanSCjD2ywyS8xqe-pBZ8/getMe
2020-03-16 10:54:33,542 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, redir

NetworkError: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot1005315252:AAH3vw40Y3WJnSWanSCjD2ywyS8xqe-pBZ8/getMe (Caused by NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x10d23ccc0>: Failed to establish a new connection: [Errno 61] Connection refused'))

## TgBot а теперь с ИИ

Концептуально нового  здесь ничего нет. по сути демонстрация и пересказ одного из туториалов выше

In [44]:
!pip install apiai

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [47]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import apiai, json
updater = Updater(token=API_TOKEN) # Токен API к Telegram
dispatcher = updater.dispatcher

/Users/biryuk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
#Суть то же самое приветствие
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')

In [49]:
API_TOKEN_FOR_AI = 'TOKEN'
client = apiai.ApiAI(API_TOKEN_FOR_AI)


По сути в нашем боте не будет никакой логики. Он будет принимать текст, оттправлять его в DialogFlow, смотреть что ответит бот там, а потом отправлять данный той машинерией ответ назад пользователю. Зато красиво.
Инструкция как получить ключ: https://habr.com/ru/post/346606/

In [50]:
def textMessage(bot, update):
    request = client.text_request() # Токен API к Dialogflow
    request.lang = 'ru' # На каком языке будет послан запрос
    request.session_id = 'HseBot' # ID Сессии диалога (нужно, чтобы потом учить бота)
    request.query = update.message.text # Посылаем запрос к ИИ с сообщением от юзера
    responseJson = json.loads(request.getresponse().read().decode('utf-8'))
    response = responseJson['result']['fulfillment']['speech'] # Разбираем JSON и вытаскиваем ответ
    # Если есть ответ от бота - присылаем юзеру, если нет - бот его не понял
    if response:
        bot.send_message(chat_id=update.message.chat_id, text=response)
    else:
        bot.send_message(chat_id=update.message.chat_id, text='Я Вас не совсем понял!')

In [ ]:
# Хендлеры
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()

2020-03-16 09:50:55,703 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x117d77cc0>: Failed to establish a new connection: [Errno 61] Connection refused')': /bot1005315252:AAH3vw40Y3WJnSWanSCjD2ywyS8xqe-pBZ8/deleteWebhook
2020-03-16 09:50:55,734 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x117d77710>: Failed to establish a new connection: [Errno 61] Connection refused')': /bot1005315252:AAH3vw40Y3WJnSWanSCjD2ywyS8xqe-pBZ8/deleteWebhook
2020-03-16 09:50:55,761 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, 

## VK API

Возможно в этом месте вы задумаетесь как же находить удобные модули. Вот например для вк. Или для твиттера. Или для инстаграмма. Спасает https://github.com/ или Google. Но первый обычно удобнее. В поиске пишем более мнее внятно и коротко что мы хотим, потом добавляем язык на котором нас это интересует (в нашем случае это python или jupiter notebook) и вуаля. Чем больше звезд - тем более популярна и признана библиотека. Использовать такие проще и потятнее, а еще у них чаще бывает внятная документация.

Популярна вот эта:
https://github.com/python273/vk_api

Хотя не менее популярна вот эта: https://github.com/voronind/vk и ее мы тоже посмотрим.

Hазницы по факту почти никакой. Для личного быстрого пользования проще авторизоваться и использовать первую. 

In [1]:
! pip install vk_api vk

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Самое сложное в начале работы с вк - получить несчастный ключ доступа. Оно и хорошо с другой стороны - этот ключ доступа дает полное право на работу от имени любого человека, все равно что знание логина и пароля. Мы обойдемся без этого. Два наиболее простых варианта - взять включ от приложения (и для этого надо самому завести приложение). Или ключ от сообщества. Оба берутся в настройках и просто копируются.

ВНИМАНИЕ: ключ доступа - все равно что логин и пароль. не передедвайте их никому

In [30]:
import vk # загружаем модуль, второй
API_TOKEN_VK = '21707fdf21707fdf21707fdf37212485c92217021707fdf7f1efcaad74eb69d965313a9'
session = vk.Session(access_token=API_TOKEN_VK)
my_api = vk.API(session, v='5.103')


Теперь у нас есть возможность использовать информацию с вк по средствам my_api - своего рода "тунеля" до вконтакте. Все интересующие функции можно найти в https://vk.com/dev/methods

Напомню. мы используем ключ доступа приложения, а потому толком не авторизовались. Мы не можем писать сообщения, публикации на стене, не можем смотреть закрытые профили, но можем смотреть на пользователей где это возможно =)

Терепрь у нас есть метод модуль который позволяет взаимодействовать с вк! погнали.
В целях эксперимента посмотрим какую нибудь информацию о пользователе. Например, список другей первого круга, второго и третьго.

In [32]:
user_to_prepare = 'vorobevana' # пользователь, с которого начнем. 

user_info = my_api.users.get(user_ids=user_to_prepare) # посмотрим что видно про пользователя
print(user_info)
start_id = user_info[0]['id']

[{'id': 174143467, 'first_name': 'Anastasia', 'last_name': 'Vorobyova', 'is_closed': False, 'can_access_closed': True}]


In [34]:
friends_first_hand = my_api.friends.get(user_id=start_id) # тот самый идентификатор
print(friends_first_hand)
friends_first_hand_id = friends_first_hand['items']

{'count': 407, 'items': [1467698, 2233698, 2594910, 4378298, 4552040, 5486806, 5801775, 6198894, 8008518, 9458352, 9891410, 9940752, 10484972, 10543902, 14162314, 14922784, 17279168, 18463599, 18572146, 20581815, 20704105, 21387930, 22495414, 22768314, 26007040, 26069773, 26358992, 27329564, 27913053, 28624978, 30484221, 30825085, 31211914, 31979264, 34018852, 34800335, 35519612, 40294783, 40865513, 44183777, 44250199, 46300439, 47531348, 47719965, 48887509, 49486124, 49641482, 50513029, 51080973, 51081096, 52380512, 53301369, 53414158, 53493744, 55889866, 57164417, 57490662, 57499188, 58903574, 60062487, 60293787, 62731697, 62819064, 63769714, 66832017, 69238678, 71100999, 73045899, 73816401, 75873015, 76723039, 76991683, 77151843, 78401573, 78473150, 80017106, 81827547, 82185380, 82715561, 82858281, 83076438, 84231303, 84440975, 86453466, 86791591, 86895494, 91681119, 92267363, 92345542, 93588459, 93873904, 94053804, 94147875, 95576514, 96918865, 97934145, 100548999, 100741742, 10170

In [35]:
# Давайте соберем пользователей второго круга!
friends_second_hand_id = []
private_accaunts = []
for item in friends_first_hand_id:
    try:
        ids = my_api.friends.get(user_id=item)['items']
        friends_second_hand_id.extend(ids)
    except:
        print('Пользователь с id={}, имеет приватный аккаунт!'.format(item))
        private_accaunts.append(item)
    
friends_second_hand_id = list(set(friends_second_hand_id))
print('На втором кругу уже {} друзей'.format(len(friends_second_hand_id)))

# на этом можно остановиться если их много

Пользователь с id=9458352, имеет приватный аккаунт!
Пользователь с id=9940752, имеет приватный аккаунт!
Пользователь с id=26007040, имеет приватный аккаунт!
Пользователь с id=28624978, имеет приватный аккаунт!
Пользователь с id=30825085, имеет приватный аккаунт!
Пользователь с id=34018852, имеет приватный аккаунт!
Пользователь с id=34800335, имеет приватный аккаунт!
Пользователь с id=49486124, имеет приватный аккаунт!
Пользователь с id=73045899, имеет приватный аккаунт!
Пользователь с id=76991683, имеет приватный аккаунт!
Пользователь с id=78473150, имеет приватный аккаунт!
Пользователь с id=82858281, имеет приватный аккаунт!
Пользователь с id=103099091, имеет приватный аккаунт!
Пользователь с id=103313375, имеет приватный аккаунт!
Пользователь с id=131981163, имеет приватный аккаунт!
Пользователь с id=132525067, имеет приватный аккаунт!
Пользователь с id=138222593, имеет приватный аккаунт!
Пользователь с id=143454335, имеет приватный аккаунт!
Пользователь с id=154065037, имеет приватн

Давайте ограничимся первым кругом и посчитаем какие нибудь статистики по друзьям.

In [37]:
freinds_to_anallize = [item for item in friends_first_hand_id if item not in private_accaunts]
#оставили только открытые аккаунты

infos = my_api.users.get(
    user_ids=freinds_to_anallize,
    fields=['sex', 'city', 'country', 'contacts', 'universities', 'relation', 'career']
)
print(infos[:3])
# к сожалению большая часть пунктов будет скрыта при поиске, но нас это не очень остановит

[{'id': 1467698, 'first_name': 'Tatyana', 'last_name': 'Komyakova', 'is_closed': False, 'can_access_closed': True, 'sex': 1, 'city': {'id': 49, 'title': 'Yekaterinburg'}, 'country': {'id': 1, 'title': 'Russia'}}, {'id': 2233698, 'first_name': 'Maria', 'last_name': 'Kalinina', 'is_closed': False, 'can_access_closed': True, 'sex': 1, 'city': {'id': 49, 'title': 'Yekaterinburg'}, 'country': {'id': 1, 'title': 'Russia'}}, {'id': 2594910, 'first_name': 'Ekaterina', 'last_name': 'Martelova', 'is_closed': False, 'can_access_closed': True, 'sex': 1, 'country': {'id': 1, 'title': 'Russia'}}]


In [38]:
import collections

# посчитаем соотношение полов:
sex = collections.Counter(item.get('sex') for item in infos)
print("Сколько людей какого пола в друзьях (2 - мужской, 1 - женский):", sex)

# и по странам: 
country = collections.Counter(item.get('country', {}).get('title') for item in infos)
print("Статистика по странам:", country)

# и по городам: 
city = collections.Counter(item.get('city', {}).get('title') for item in infos)
print("Статистика по городам:", city)

Сколько людей какого пола в друзьях (2 - мужской, 1 - женский): Counter({1: 216, 2: 125})
Статистика по странам: Counter({'Russia': 294, None: 38, 'USA': 2, 'Czech Republic': 2, 'Israel': 1, 'Germany': 1, 'United Arab Emirates': 1, 'Kazakhstan': 1, 'Brazil': 1})
Статистика по городам: Counter({'Yekaterinburg': 162, 'Moscow': 70, None: 69, 'Saint Petersburg': 13, 'Kazan': 2, 'Praha': 2, 'Berezovsky': 2, 'Ivanovo': 1, 'Magnitogorsk': 1, 'Dolgoprudny': 1, 'Zarechny': 1, 'Rishon LeZion': 1, 'Monchegorsk': 1, 'Lesnoy': 1, 'Chelyabinsk': 1, 'Rybinsk': 1, 'Kirov': 1, 'Pskov': 1, 'Orenburg': 1, 'Karlsruhe': 1, 'Ivdel': 1, 'Petrozavodsk': 1, 'Dvurechensk': 1, 'Dubai': 1, 'Vladivostok': 1, 'Almaty': 1, 'Arkhangelsk': 1, 'Portland': 1})


In [39]:
for user in infos:
    if user.get('city', {}).get('title') == 'Lesnoy':
        print(user)

{'id': 114204934, 'first_name': 'Polina', 'last_name': 'Khokhlova', 'is_closed': False, 'can_access_closed': True, 'sex': 1, 'city': {'id': 11671, 'title': 'Lesnoy'}, 'country': {'id': 1, 'title': 'Russia'}, 'mobile_phone': '', 'home_phone': '', 'career': [], 'relation': 0, 'universities': [{'id': 526, 'country': 1, 'city': 11671, 'name': 'КНИТУ (бывш. КГТУ, КХТИ)', 'faculty': 7833, 'faculty_name': 'Инженерный химико-технологический институт', 'chair': 47808, 'chair_name': ' Кафедра «Технологии твердых химических веществ» (ФЭМИ)'}]}


In [40]:
# Соберем последние записи со стен друзей
posts = []
for item in freinds_to_anallize:
    try:
        posts.append(my_api.wall.get(
            owner_id=item,
            count=10, #ограничимся последним 10 записями
            filter='owner', # берем только записи от самого пользователя
            extended=1,
        ))
    except:
        print('User {} hide his wall'.format(item))
print(posts[:2])

User 4378298 hide his wall
User 5486806 hide his wall
User 9891410 hide his wall
User 21387930 hide his wall
User 26069773 hide his wall
User 26358992 hide his wall
User 27329564 hide his wall
User 27913053 hide his wall
User 31211914 hide his wall
User 40294783 hide his wall
User 40865513 hide his wall
User 47531348 hide his wall
User 47719965 hide his wall
User 48887509 hide his wall
User 51080973 hide his wall
User 57164417 hide his wall
User 57490662 hide his wall
User 57499188 hide his wall
User 60062487 hide his wall
User 62819064 hide his wall
User 63769714 hide his wall
User 69238678 hide his wall
User 73816401 hide his wall
User 77151843 hide his wall
User 78401573 hide his wall
User 81827547 hide his wall
User 82185380 hide his wall
User 86791591 hide his wall
User 95576514 hide his wall
User 102992733 hide his wall
User 103190352 hide his wall
User 103605329 hide his wall
User 104857494 hide his wall
User 104949205 hide his wall
User 105103519 hide his wall
User 106652617 hi

Вместо послесловия:

Работа с личным ключем может дать заметно больше информации и простора для аналитики. большая часть профилей логично скрыта от такого обхода.

В рамках самостоятельного исследования можете обратить внимание еще на функционал групп, который позволяет аналогично так же строить диалоговые системы от имени группы.

А так же можете обратить внимания на методы связанные с получаением лайков, коментов и репостов, там тоже много полезного.